Views look like this in their simplified string notation:

In [2]:
"{P(a())Q(b())R(c())}^{0}"

'{P(a())Q(b())R(c())}^{0}'

Instead of operating on `View` objects directly, let's consider whether certain mutation
operations could be done on the string representations with less overhead.

In [3]:
stage = "{P(a())Q(b())R(c())}^{0}".split("}^{")[0][1:]
stage

'P(a())Q(b())R(c())'

In [4]:
from pyetr import View
v = View.from_str("{P(a())Q(b())R(c())}^{0}")
str(v.stage)

'{Q(b)P(a)R(c)}'

In [5]:
list(list(v.stage)[0])

[Q(b), P(a), R(c)]

So, I don't think it'll be pure string parsing. But, we can disassemble and reassemble
views using their `.stage` and `.supposition` attributes, then add in or mutate the
decomposed string representations before adding any new information (predicates or
quantifiers) _as strings_—much simpler than actually constructing `PredicateAtom`s or
others from scratch.

_NOTE_: taking this back. When you print `.stage` you for some reason lose the
parentheses identifying constants as constants and not arbitrary. So, I think we're just
going to split on `}^{`. Or, maybe more versatile, we'll just process with
`[:-1] + "())"` whenever we print stages.

In [6]:
import re
print(re.findall("[A-Z]", str(v.atoms)))
print(re.findall("[a-z]", str(v.atoms)))
max_object = max(re.findall("[a-z]", str(v.atoms)))

print(chr(ord(max_object) + 1))

['Q', 'P', 'R']
['b', 'a', 'c']
d


In [7]:
str(list(v.stage)[0]) + 

SyntaxError: invalid syntax (2638085100.py, line 1)

In [8]:
def atom_to_str(atom) -> str:
    return str(atom)[:-1] + "())"

def state_to_str(state) -> str:
    return "".join([atom_to_str(a) for a in state])

def set_of_states_to_str(set_of_states) -> str:
    return "{" + ",".join([state_to_str(s) for s in set_of_states]) + "}"

In [9]:
print([atom_to_str(a) for a in v.atoms])
print(v.atoms)
print(set_of_states_to_str(v.stage))
print(v)

['Q(b())', 'P(a())', 'R(c())']
{Q(b), P(a), R(c)}
{Q(b())P(a())R(c())}
{Q(b())P(a())R(c())}


In [10]:
from typing import cast
from pyetr import ArbitraryObject, FunctionalTerm, PredicateAtom

def get_object_sets_for_view(
    view: View
) -> tuple[dict[str, set[str]], set[str], set[str]]:
    """Take a view and return sets of predicates (at different arities), constants, and
        arbitrary objects used in the view.

    Args:
        view (View): The View to parse.

    Raises:
        ValueError: If we ever handle a FunctionalTerm with a nonzero arity.
        ValueError: If we ever handle a term that's neither an ArbitraryObject nor a
            Functional Term

    Returns:
        tuple[dict[str, set[str]], set[str], set[str]]:
            dict[str, set[str]]: Predicates, mapping arity to a set of letters
            set[str]: Constants
            set[str]: Arbitrary objects
    """
    predicates: dict[str, set[str]] = {}
    constants: set[str] = set()
    arb_objs: set[str] = set()

    for state in view.weights._weights.keys():
        for atom in state:
            atom = cast(PredicateAtom, atom)
            arity = str(atom.predicate.arity)
            name = str(atom.predicate.name)
            if arity not in predicates.keys():
                predicates[arity] = set([name])
            else:
                predicates[arity].add(name)

            for term in atom.terms:
                if type(term) == ArbitraryObject: arb_objs.add(term.name)
                elif type(term) == FunctionalTerm:
                    if term.f.arity != 0: raise ValueError
                    constants.add(term.f.name)
                else: raise ValueError

    return predicates, constants, arb_objs

In [43]:
def get_view_mutations(view: View) -> set[View]:
    mutations = set()

    # Assemble lists of strings of predicates, constants, and arbitrary objects
    predicates, constants, arb_objs = get_object_sets_for_view(view)

    if len(predicates.keys()) > 1: raise ValueError
    predicates = predicates["1"]  # For now we only do unary predicates

    max_predicate = max(predicates)
    new_predicate = chr(ord(max_predicate) + 1)

    if len(arb_objs) == 0:
        arb_objs = set(["a"])
        new_arb_obj = "a"
    else:
        max_arb_obj = max(arb_objs)
        new_arb_obj = chr(ord(max_arb_obj) + 1)

    for o in constants:
        # Also append problems where the variable is taken to be at issue in all
        # occurrences (can I do this with issue only occurring sometimes?)
        for is_at_issue in [new_arb_obj, new_arb_obj + "*"]:
            mutations.add(
                View.from_str(
                    f"A{new_arb_obj} " + view.to_str().replace(o + "()", is_at_issue)
                )
            )
            mutations.add(
                View.from_str(
                    f"E{new_arb_obj} " + view.to_str().replace(o + "()", is_at_issue)
                )
            )

    if len(constants) == 0:
        constants = set(["a"])
        new_constant = "a"
    else:
        max_constant = max(constants)
        new_constant = chr(ord(max_constant) + 1)

    predicates.add(new_predicate)
    constants.add(new_constant)

    atoms = [atom_to_str(a) for a in view.atoms] + [f"{p}({o}())" for p in predicates for o in constants]

    for atom in atoms:
        atom_at_issue = atom[:-1] + "*)"
        # All conjunctions
        for i in range(len(list(view.stage))):
            new_state_str = state_to_str(list(view.stage)[i]) + atom
            new_stage_str = "{" + ",".join(
                list([state_to_str(s) for s in view.stage])[:i] +
                [new_state_str] +
                list([state_to_str(s) for s in view.stage])[i+1:]
            ) + "}"
            mutations.add(View.from_str(new_stage_str))

            # Also add one where the new atom is at issue
            new_state_str = state_to_str(list(view.stage)[i]) + atom_at_issue
            new_stage_str = "{" + ",".join(
                list([state_to_str(s) for s in view.stage])[:i] +
                [new_state_str] +
                list([state_to_str(s) for s in view.stage])[i+1:]
            ) + "}"
            mutations.add(View.from_str(new_stage_str))
        
        # Disjunctions
        mutations.add(
            View.from_str(
                "{" + ",".join(
                list([state_to_str(s) for s in view.stage]) +
                [atom]
            ) + "}"
            )
        )
        mutations.add(
            View.from_str(
                "{" + ",".join(
                list([state_to_str(s) for s in view.stage]) +
                [atom_at_issue]
            ) + "}"
            )
        )

    return mutations

get_view_mutations(View.from_str("∀a {A(a)A(a())}"))


{{A(a()),A(b())},
 {A(a()),A(b()*)},
 {A(a()),B(a())},
 {A(a()),B(a()*)},
 {A(a()),B(b())},
 {A(a()),B(b()*)},
 {A(a())B(b())},
 {A(a())B(b()*)},
 {A(a())},
 {A(a()*)},
 {A(b())A(a())},
 {A(b()*)A(a())},
 {B(a())A(a())},
 {B(a()*)A(a())},
 ∀b ∀a {A(b)A(a)},
 ∀b ∀a {A(b*)A(a)},
 ∃b ∀a {A(b)A(a)},
 ∃b ∀a {A(b*)A(a)}}

In [12]:
def get_problem_mutations(problem: tuple[View, ...]) -> set[tuple[View, ...]]:
    mutations = set()
    for i, view in enumerate(problem):
        for mut in get_view_mutations(view):
            mutations.add(
                problem[:i] + (mut,) + problem[i+1:]
            )
    mutations.add(problem + (View.from_str("{A(a())}"),))
    return mutations

In [13]:
from etr_case_generator.reasoning_problem import ReasoningProblem

def mutate_reasoning_problem(r: ReasoningProblem) -> set[ReasoningProblem]:
    if len(r.premises) > 0 and random.random() < 0.5:
        # Mutate one of the existing premises
        i = random.randint(0, len(r.premises) - 1)
        premises = [p[0] for p in r.premises]
        premises[i] = mutate_view(premises[i])
        r.update_premises(premises=premises)
        return r
    elif len(r.premises) <= max_premises and random.random() < 0.5:
        # Add a new premise
        r.update_premises(premises=[p[0] for p in r.premises] + [View.get_falsum()])
        return r
    else:
        # Mutate query
        r.update_query(query=mutate_view(r.query[0]))
        return r

def ignore_problem(r: ReasoningProblem) -> bool:
    if len(r.premises) == 0:
        return True
    return len(r.premises) <= 1 and r.premises[0][0].is_falsum

problem_queue = [ReasoningProblem(generator=ETRCaseGenerator(ELEMENTS))]    

NameError: name 'ETRCaseGenerator' is not defined

In [14]:
# In this iteration, a reasoning problem is a tuple where the final element is the query
reasoning_problems: set[tuple[View, ...]] = set([(View.from_str("{A(a())}"), View.from_str("{A(a())}"))])

In [15]:
from etr_case_generator.ontology import ELEMENTS
from pyetr.inference import default_inference_procedure
from etr_case_generator.view_to_natural_language import view_to_natural_language

from time import sleep

from etr_case_generator.generator import ETRCaseGenerator
from etr_case_generator.reasoning_problem import ReasoningProblem


while len(reasoning_problems) > 0:
    problem = reasoning_problems.pop()

    # Format current problem as a ReasoningProblem and write it out
    r = ReasoningProblem(generator=ETRCaseGenerator(ELEMENTS))
    r.update_premises(list(problem[:-1]))
    r.update_query(problem[-1])
    d = r.to_dict()
    print(d["full_prose"])
    print()
    print(f"etr_conclusion: {d['etr_conclusion']}")
    print(f"etr_conclusion_is_categorical: {d['etr_conclusion_is_categorical']}")

    for mut in get_problem_mutations(problem):
        for i, view in enumerate(mut):
            reasoning_problems.add(mut)
    print(len(reasoning_problems))

Consider the following premises:

1. Mystarium is transparent to visible light.

Does it follow that mystarium is transparent to visible light?

Answer using 'YES' or 'NO' ONLY.

etr_conclusion: ('{0}', '')
etr_conclusion_is_categorical: False
18
Consider the following premises:

1. Phantasmite is gravity-enhancing.

Does it follow that fluxium is gravity-enhancing and phantasmite is gravity-enhancing?

Answer using 'YES' or 'NO' ONLY.

etr_conclusion: ('{0}', '')
etr_conclusion_is_categorical: False
41
Consider the following premises:

1. Aurorium is emotion-reactive.

Does it follow that aurorium is volatile and aurorium is emotion-reactive?

Answer using 'YES' or 'NO' ONLY.

etr_conclusion: ('{0}', '')
etr_conclusion_is_categorical: False
61
Consider the following premises:

1. Either fluxium is thermal-conductive, or pyroflux is plasma-like.

Does it follow that fluxium is thermal-conductive?

Answer using 'YES' or 'NO' ONLY.

etr_conclusion: ('{0}', '')
etr_conclusion_is_categoric

KeyboardInterrupt: 

In [46]:
view_to_natural_language(ELEMENTS, View.from_str("∀b ∀a {A(b*)A(a)}"))

('for all B, for all A, B is gaseous under high pressure and A is gaseous under high pressure',
 {'A': 'gaseous under high pressure'})

In [19]:
type(reasoning_problems)

set

In [20]:
import json

with open(f"datasets/queue_iteration.jsonl", "w") as f:
    for problem in reasoning_problems:
        r = ReasoningProblem(generator=ETRCaseGenerator(ELEMENTS))
        r.update_premises(list(problem[:-1]))
        r.update_query(problem[-1])
        f.write(json.dumps(r.to_dict()) + "\n")

SolverReturnedUnknownResultError: 

In [117]:
print(View.from_str("∃x {A(x),A(x())}").detailed)

<View
  stage={{<PredicateAtom predicate=<Predicate name=A arity=1> terms=(<FunctionalTerm f=Function(x, 0) t=()>)>},{<PredicateAtom predicate=<Predicate name=A arity=1> terms=(<ArbitraryObject name=x>)>}}
  supposition={{}}
  dep_rel=<DependencyRelation deps=[] unis=frozenset() exis=frozenset({x})>
  issue_structure={}
  weights=<Weights {<PredicateAtom predicate=<Predicate name=A arity=1> terms=(<ArbitraryObject name=x>)>}: <Weight multi=<Multiset items=[]> add=<Multiset items=[]>>,{<PredicateAtom predicate=<Predicate name=A arity=1> terms=(<FunctionalTerm f=Function(x, 0) t=()>)>}: <Weight multi=<Multiset items=[]> add=<Multiset items=[]>>>
>


In [142]:
for k in View.from_str("∃x {A(x)B(x),A(x())}").weights._weights.keys():
    for a in k:
        print(a.terms[0].detailed)

<ArbitraryObject name=x>
<ArbitraryObject name=x>
<FunctionalTerm f=Function(x, 0) t=()>


In [39]:
default_inference_procedure([View.from_str("Ax {A(x)}^{B(x*)}"), View.from_str("{B(a()*)}")])

{A(a())}

In [42]:
View.from_str("{B(a())B(a()*)}")

{B(a()*)}

In [89]:
re.findall(r"[A-Z]", v.to_str())


['P', 'R', 'Q']

In [149]:
# Assemble lists of strings of predicates, constants, and arbitrary objects
predicates: dict[str, set[str]] = {}
constants: set[str] = set()
arb_objs: set[str] = set()

for state in View.from_str("∃x {A(x)B(x),A(x())}").weights._weights.keys():
    for atom in state:
        atom = cast(PredicateAtom, atom)
        arity = str(atom.predicate.arity)
        name = str(atom.predicate.name)
        if arity not in predicates.keys():
            predicates[arity] = set([name])
        else:
            predicates[arity].add(name)

        for term in atom.terms:
            if type(term) == ArbitraryObject: arb_objs.add(term.name)
            elif type(term) == FunctionalTerm:
                if term.f.arity != 0: raise ValueError
                constants.add(term.f.name)
            else: raise ValueError